In [1]:
%cd ..

/Users/Matteo/PycharmProjects/TrafficEmu


# Bayesian optimization: finding the minimum emission

In this notebook we will performing bayesian optimization on the emulator trained on our sumo simulation. We are interested in identifying the parameters which enable us to obtain the minimum travel time. Our emulator is a gaussian process with an **rbf** kernel.


## Imports

In [7]:
import pickle
import numpy as np
import emukit as ek
import GPy
import pandas as pd

from emukit.model_wrappers import GPyModelWrapper
from emukit.core.initial_designs import RandomDesign
from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop

from sumo_grid_simulation.grid_simulation import Simulator
from experimental_design import config

## Simulator

We start by initialising our simulation.

In [8]:
simulator = Simulator(end_time=300)

We then define the two user functions in which we are interested:

In [13]:
def user_function_emissions(X):
    """  X = inputs - emukit doesnt pass named args, just an NxM ndarray, N is the number of points to evaluate, M is the number of parameters per each point """
    result = []
    i = 0
    
    print(X)
    print(f'\nUser function emissions called with {X.shape[0]} inputs to simulate')

    
    for gridSize, edgeMaxSpeed, edgeLength, numLanes, accel in X:
        print(f'\nEvaluating input: {i+1} of {X.shape[0]}\n')
        
        beta = 0.05
        max_number_of_vehicles = ((gridSize - 1) * gridSize * 2 + 4 * gridSize) * edgeLength / 5
        period = 300/(max_number_of_vehicles * beta)
        
        s = simulator.simulate(
            gridSize      = int(gridSize),
            edgeMaxSpeed  = edgeMaxSpeed,
            edgeLength    = edgeLength,
            numberOfLanes = int(numLanes),
            accel         = accel,
            trips_generator_period = period
        )
        

        CO2 = s['CO2']/s['num_emissions_samples']
        result.append(CO2)
        print(f'\nOutput {result[-1]}\n')
        i += 1
        
    # expand dims is essential or the acquition function breaks
    return np.expand_dims(np.array(result), 1)  

We then load the parameters' space from our configuration file:

In [14]:
parameter_space = config.get_parameter_space()

## Emulator

### Rnadomly initialised emulator

The first emulator we are going to analyse is the one that is just initilised with random points

We first sample at random 200 datapoints from the parameter space:

In [15]:
#init_X, init_Y = pickle.load(open('bayesian_optimization/init_points/270_init_points_emissions.pkl', 'rb'))


design = RandomDesign(parameter_space)
num_init_points = 520
init_X = design.get_samples(num_init_points)
init_Y = user_function_emissions(init_X)
init_points = init_X, init_Y
with open(f'bayesian_optimization/init_points/{num_init_points}_init_points_emissions.pkl', "wb") as f:
     pickle.dump(init_points, f)


[[ 4.         14.15010191 55.84683916  1.          4.79832242]
 [14.         15.57041614 46.57856287  2.          4.64578317]
 [ 6.         12.80759205 56.07463308  1.          2.06556408]
 ...
 [18.         16.37192794 60.15180318  2.          1.68195998]
 [ 9.          8.74817599 50.2642949   3.          2.02774525]
 [14.         12.4840758  47.95369231  3.          2.67440027]]

User function emissions called with 520 inputs to simulate

Evaluating input: 1 of 520

 Retrying in 1 seconds

Output 4861.245391705068


Evaluating input: 2 of 520

 Retrying in 1 seconds

Output 4511.384151717974


Evaluating input: 3 of 520

 Retrying in 1 seconds

Output 3271.347444286389


Evaluating input: 4 of 520

 Retrying in 1 seconds

Output 3499.1660758293847


Evaluating input: 5 of 520

 Retrying in 1 seconds

Output 4122.023564642357


Evaluating input: 6 of 520

 Retrying in 1 seconds

Output 3495.2498501512205


Evaluating input: 7 of 520

 Retrying in 1 seconds

Output 3458.010946745562





Output 3178.8590843621446


Evaluating input: 98 of 520

 Retrying in 1 seconds

Output 2351.462857142855


Evaluating input: 99 of 520

 Retrying in 1 seconds

Output 2373.598487261148


Evaluating input: 100 of 520

 Retrying in 1 seconds

Output 2927.6106303084443


Evaluating input: 101 of 520

 Retrying in 1 seconds

Output 2032.5512450240096


Evaluating input: 102 of 520

 Retrying in 1 seconds

Output 3825.9932528070444


Evaluating input: 103 of 520

 Retrying in 1 seconds

Output 4952.3093945270975


Evaluating input: 104 of 520

 Retrying in 1 seconds

Output 4548.3677188815755


Evaluating input: 105 of 520

 Retrying in 1 seconds

Output 3390.0144728533087


Evaluating input: 106 of 520

 Retrying in 1 seconds

Output 5146.968696128564


Evaluating input: 107 of 520

 Retrying in 1 seconds

Output 2393.968950574384


Evaluating input: 108 of 520

 Retrying in 1 seconds

Output 2662.3221485333675


Evaluating input: 109 of 520

 Retrying in 1 seconds

Output 3612.274224452


Output 4242.921919111825


Evaluating input: 199 of 520

 Retrying in 1 seconds

Output 2896.1631725465


Evaluating input: 200 of 520

 Retrying in 1 seconds

Output 3019.6049064764825


Evaluating input: 201 of 520

 Retrying in 1 seconds

Output 4623.586606217617


Evaluating input: 202 of 520

 Retrying in 1 seconds

Output 5204.933543543539


Evaluating input: 203 of 520

 Retrying in 1 seconds

Output 3603.8871464019853


Evaluating input: 204 of 520

 Retrying in 1 seconds

Output 3813.7786027568955


Evaluating input: 205 of 520

 Retrying in 1 seconds

Output 2578.3906113537096


Evaluating input: 206 of 520

 Retrying in 1 seconds

Output 5228.877596671556


Evaluating input: 207 of 520

 Retrying in 1 seconds

Output 5493.074543985879


Evaluating input: 208 of 520

 Retrying in 1 seconds

Output 4328.723071339819


Evaluating input: 209 of 520

 Retrying in 1 seconds

Output 3216.056571112684


Evaluating input: 210 of 520

 Retrying in 1 seconds

Output 4795.323545150498


 Retrying in 1 seconds

Output 4024.5002170292178


Evaluating input: 300 of 520

 Retrying in 1 seconds

Output 4469.572397476343


Evaluating input: 301 of 520

 Retrying in 1 seconds

Output 3719.348575342465


Evaluating input: 302 of 520

 Retrying in 1 seconds

Output 3577.602180038191


Evaluating input: 303 of 520

 Retrying in 1 seconds

Output 3253.181257142855


Evaluating input: 304 of 520

 Retrying in 1 seconds

Output 2335.026243902443


Evaluating input: 305 of 520

 Retrying in 1 seconds

Output 4721.337777508623


Evaluating input: 306 of 520

 Retrying in 1 seconds

Output 3287.711804697713


Evaluating input: 307 of 520

 Retrying in 1 seconds

Output 3762.3140225431366


Evaluating input: 308 of 520

 Retrying in 1 seconds

Output 3070.34266917293


Evaluating input: 309 of 520

 Retrying in 1 seconds

Output 2754.7364376590317


Evaluating input: 310 of 520

 Retrying in 1 seconds

Output 5202.562447165721


Evaluating input: 311 of 520

 Retrying in 1 seconds

Ou

 Retrying in 1 seconds

Output 4119.731954031903


Evaluating input: 401 of 520

 Retrying in 1 seconds

Output 3703.105784237209


Evaluating input: 402 of 520

 Retrying in 1 seconds

Output 4652.3442815249255


Evaluating input: 403 of 520

 Retrying in 1 seconds

Output 2217.7139577969174


Evaluating input: 404 of 520

 Retrying in 1 seconds

Output 2184.8293149259116


Evaluating input: 405 of 520

 Retrying in 1 seconds

Output 4668.621301836667


Evaluating input: 406 of 520

 Retrying in 1 seconds

Output 3717.1579406220612


Evaluating input: 407 of 520

 Retrying in 1 seconds

Output 3427.79356634077


Evaluating input: 408 of 520

 Retrying in 1 seconds

Output 3569.828966241313


Evaluating input: 409 of 520

 Retrying in 1 seconds

Output 3542.3212256546767


Evaluating input: 410 of 520

 Retrying in 1 seconds

Output 3856.572153284676


Evaluating input: 411 of 520

 Retrying in 1 seconds

Output 3982.097435451844


Evaluating input: 412 of 520

 Retrying in 1 seconds



 Retrying in 1 seconds

Output 3608.6297959104427


Evaluating input: 502 of 520

 Retrying in 1 seconds

Output 2394.942411593402


Evaluating input: 503 of 520

 Retrying in 1 seconds

Output 2890.0878223708637


Evaluating input: 504 of 520

 Retrying in 1 seconds

Output 2407.3551109139344


Evaluating input: 505 of 520

 Retrying in 1 seconds

Output 3825.4421404602085


Evaluating input: 506 of 520

 Retrying in 1 seconds

Output 3013.5238830383146


Evaluating input: 507 of 520

 Retrying in 1 seconds

Output 2238.9588254936134


Evaluating input: 508 of 520

 Retrying in 1 seconds

Output 3589.9906201096765


Evaluating input: 509 of 520

 Retrying in 1 seconds

Output 3701.3345806945


Evaluating input: 510 of 520

 Retrying in 1 seconds

Output 2258.341162690274


Evaluating input: 511 of 520

 Retrying in 1 seconds

Output 4859.65384269663


Evaluating input: 512 of 520

 Retrying in 1 seconds

Output 4419.020348101258


Evaluating input: 513 of 520

 Retrying in 1 seconds



To then fit a GP to these points

In [16]:
emulator = GPy.models.GPRegression(init_X, init_Y, noise_var=1e-10)
emukit_model_random_init = GPyModelWrapper(emulator, n_restarts=5)
emukit_model_random_init.optimize()

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 1/5, f = 24586.735957959267
Optimization restart 2/5, f = 3104.9173509798766
Optimization restart 3/5, f = 6790.620458906006
Optimization restart 4/5, f = 3086.0019231072765
Optimization restart 5/5, f = 3085.875237865033


### Experimentally designed emulator with model variance

We also load the emulator obtained during experimental design:

In [17]:
emukit_model_variance = pickle.load(open('experimental_design/models/ms_5param_model_variance_20_init_points_500_loops_co2_per_duration.pkl', 'rb'))

### Experimentally designed emulator with integrated variance reduction

We also load the emulator obtained during experimental design:

In [18]:
emukit_model_integrated_variance = pickle.load(open('experimental_design/models/ms_5param_integrated_variance_reduction_20_init_points_500_loops_co2_per_duration.pkl', 'rb'))

## Bayesian optimisation

We now run bayesian optimisation on the emulators

In [19]:
n_iter_bo = 50

### Acquisition functions

In [20]:
acquisition_random_init = ExpectedImprovement(emukit_model_random_init)

In [21]:
acquisition_model_variance = ExpectedImprovement(emukit_model_variance)

In [22]:
acquisition_integrated_variance = ExpectedImprovement(emukit_model_integrated_variance)

### Optimisation loop

In [23]:
bo_random_init = BayesianOptimizationLoop(parameter_space, emukit_model_random_init, acquisition=acquisition_random_init)
bo_random_init.run_loop(user_function_emissions, n_iter_bo)

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 3085.875237865033
Optimization restart 2/5, f = 4357.285512280114
Optimization restart 3/5, f = 3097.704927760546
Optimization restart 4/5, f = 3818.6497640896687
Optimization restart 5/5, f = 3240.1244522961847
[[16.  8. 70.  1.  5.]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 2046.3772749496873



 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 3107.8351761718886
Optimization restart 2/5, f = 3106.9833153547083
Optimization restart 3/5, f = 3106.984520571412


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


...

Optimization restart 1/5, f = 3381.134655986939


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 2/5, f = 3381.1337695520247


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 3/5, f = 3381.7738323973795
Optimization restart 4/5, f = 3381.133772796604
Optimization restart 5/5, f = 3388.28611793666


In [24]:
bo_model_variance = BayesianOptimizationLoop(parameter_space, emukit_model_variance, acquisition=acquisition_model_variance)
bo_model_variance.run_loop(user_function_emissions, n_iter_bo)

Optimization restart 1/5, f = 2932.1184404617716
Optimization restart 2/5, f = 2932.1431272440227
Optimization restart 3/5, f = 2937.248586249671
Optimization restart 4/5, f = 2937.248586253984
Optimization restart 5/5, f = 2937.248586250742
[[14.          8.45953658 63.94908114  1.          1.9438681 ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 1859.836003690793



 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 2946.056239482815
Optimization restart 2/5, f = 2945.9498722598605
Optimization restart 3/5, f = 2945.759725660637
Optimization restart 4/5, f = 2945.94355181051


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 5/5, f = 3662.0790115098375
[[15.          8.         65.73244627  1.          1.5       ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 1728.5867178738658



...

Optimization restart 1/5, f = 3223.1352254374806
Optimization restart 2/5, f = 3223.1346016157277


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 3/5, f = 3223.13460222931
Optimization restart 4/5, f = 3223.134601595537
Optimization restart 5/5, f = 3223.1346015908857
[[15.          8.         66.22197708  1.          1.5       ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 1723.7617333840706



 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 3227.9953760793906
Optimization restart 2/5, f = 3227.994652058719
Optimization restart 3/5, f = 3227.9946522227433
Optimization restart 4/5, f = 3227.994652052437
Optimization restart 5/5, f = 3227.9946520574417


In [25]:
bo_integrated_variance = BayesianOptimizationLoop(parameter_space, emukit_model_integrated_variance, acquisition=acquisition_integrated_variance)
bo_integrated_variance.run_loop(user_function_emissions, n_iter_bo)

Optimization restart 1/5, f = 2872.31572980932
Optimization restart 2/5, f = 2872.3157298039346
Optimization restart 3/5, f = 2872.3157324051763
Optimization restart 4/5, f = 2872.315729809092


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 5/5, f = 2872.3157298117976
[[18.   8.  70.   1.   1.5]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 1678.9463145100683



 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 2886.629894738568


...

Optimization restart 4/5, f = 3313.612416464289
Optimization restart 5/5, f = 3173.151518416793
[[20.          8.         69.91418364  1.          1.5       ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 1663.1325559684565



 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 3177.843922898459
Optimization restart 2/5, f = 3177.8432001177503
Optimization restart 3/5, f = 3177.8432001091073
Optimization restart 4/5, f = 3177.8432001086503


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 5/5, f = 3582.8200898044383


## Results

In [26]:
parameters_name = ['Size of the grid', 'Speed limit in the net (m/s)', 'Length of the roads (m)', 'Number of lanes', 'Accelleration (m/s^2)']

### Results on the random initialised emulator

In [27]:
results_random_init = bo_random_init.get_results()

In [28]:
results_random_init_df = pd.DataFrame(results_random_init.minimum_location.reshape(1,-1))
results_random_init_df.columns = parameters_name
results_random_init_df

,Size of the grid,Speed limit in the net (m/s),Length of the roads (m),Number of lanes,Accelleration (m/s^2)
0,20.0,8.0,70.0,1.0,1.5


In [29]:
results_random_init.minimum_value

1662.044947592042

### Results on the experimentally designed emulator with model variance

In [30]:
results_model_variance = bo_model_variance.get_results()

In [31]:
results_random_init_df = pd.DataFrame(results_model_variance.minimum_location.reshape(1,-1))
results_random_init_df.columns = parameters_name
results_random_init_df

,Size of the grid,Speed limit in the net (m/s),Length of the roads (m),Number of lanes,Accelleration (m/s^2)
0,15.0,8.0,66.213268,1.0,1.5


In [32]:
results_model_variance.minimum_value

1718.5519022665358

### Results on the experimentally designed emulator with integrated variance reduction

In [33]:
results_integrated_variance = bo_integrated_variance.get_results()

In [34]:
results_random_init_df = pd.DataFrame(results_integrated_variance.minimum_location.reshape(1,-1))
results_random_init_df.columns = parameters_name
results_random_init_df

,Size of the grid,Speed limit in the net (m/s),Length of the roads (m),Number of lanes,Accelleration (m/s^2)
0,20.0,8.0,69.176962,1.0,1.5


In [35]:
results_integrated_variance.minimum_value

1661.2993951063625

## Save stuff

In [36]:
with open(f'bayesian_optimization/bayesian_opt_results/results_random_init_emissions.pkl', "wb") as f:
     pickle.dump(results_random_init, f)

with open(f'bayesian_optimization/bayesian_opt_results/results_model_variance_emissions.pkl', "wb") as f:
     pickle.dump(results_model_variance, f)
        
with open(f'bayesian_optimization/bayesian_opt_results/results_integrated_variance_emissions.pkl', "wb") as f:
     pickle.dump(results_integrated_variance, f)

In [37]:
with open(f'bayesian_optimization/models/520_random_init_50_bay_opt_emissions.pkl', "wb") as f:
     pickle.dump(emukit_model_random_init, f)

with open(f'bayesian_optimization/models/520_model_variance_50_bay_emissions.pkl', "wb") as f:
     pickle.dump(emukit_model_variance, f)
        
with open(f'bayesian_optimization/models/520_integrated_variance_50_bay_emissions.pkl', "wb") as f:
     pickle.dump(emukit_model_integrated_variance, f)